In [94]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

In [2]:
df=pd.read_csv('Baza customer Telecom v2.csv')
df

,PID,CRM_PID_Value_Segment,EffectiveSegment,Billing_ZIP,KA_name,Active_subscribers,Not_Active_subscribers,Suspended_subscribers,Total_SUBs,AvgMobileRevenue,AvgFIXRevenue,TotalRevenue,ARPU,CHURN
0,123759242,Bronze,SOHO,6000.0,VM,2,NaN,NaN,2,40.17,0.0,40.17,NaN,No
1,126145737,Bronze,SOHO,6400.0,VM,3,NaN,NaN,3,40.17,0.0,40.17,13.39,No
2,123506355,Bronze,SOHO,6000.0,DI,2,3.0,NaN,5,40.17,0.0,40.17,20.09,No
3,112595585,Bronze,SOHO,4400.0,MT,1,2.0,NaN,3,40.17,0.0,40.17,40.17,No
4,115097935,Iron,SOHO,4000.0,AD,2,1.0,NaN,3,40.17,0.0,40.17,20.09,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8448,115901902,Iron,SOHO,4000.0,VU,2,1.0,NaN,3,56.50,0.0,56.50,28.25,No
8449,115800273,Bronze,SOHO,4006.0,VU,5,NaN,NaN,5,56.50,0.0,56.50,11.30,No
8450,160049889,Bronze,SOHO,4003.0,VU,3,1.0,NaN,4,56.50,0.0,56.50,18.83,No
8451,160095203,Bronze,SOHO,4023.0,VU,3,NaN,NaN,3,56.50,0.0,56.50,18.83,Yes


In [3]:
df.isnull().sum()

PID                          0
CRM_PID_Value_Segment        5
EffectiveSegment             0
Billing_ZIP                  2
KA_name                      0
Active_subscribers           0
Not_Active_subscribers    4149
Suspended_subscribers     8101
Total_SUBs                   0
AvgMobileRevenue             0
AvgFIXRevenue                0
TotalRevenue                 0
ARPU                         1
CHURN                        0
dtype: int64

In [4]:
df.duplicated().sum()

np.int64(0)

In [5]:
print('Shape:',df.shape)

Shape: (8453, 14)


In [6]:
df.columns

Index(['PID', 'CRM_PID_Value_Segment', 'EffectiveSegment', 'Billing_ZIP',
       'KA_name', 'Active_subscribers', 'Not_Active_subscribers',
       'Suspended_subscribers', 'Total_SUBs', 'AvgMobileRevenue ',
       'AvgFIXRevenue', 'TotalRevenue', 'ARPU', 'CHURN'],
      dtype='object')

In [7]:
print(df['CHURN'].unique())

['No' 'Yes']


In [8]:
df['CHURN']= df['CHURN'].map({'Yes':1, 'No':0})
df['CHURN']

0       0
1       0
2       0
3       0
4       0
       ..
8448    0
8449    0
8450    0
8451    1
8452    0
Name: CHURN, Length: 8453, dtype: int64

In [9]:
[col for col in df.columns if 'CHURN' in col]

['CHURN']

In [10]:
corr= df.corr(numeric_only=True)['CHURN'].sort_values(ascending=False)
print(corr)

CHURN                     1.000000
AvgMobileRevenue          0.065054
TotalRevenue              0.063745
Active_subscribers        0.061711
Total_SUBs                0.045348
ARPU                      0.006476
Not_Active_subscribers    0.001157
Suspended_subscribers    -0.004451
AvgFIXRevenue            -0.007744
Billing_ZIP              -0.009134
Name: CHURN, dtype: float64


In [11]:
df.drop(columns=['Not_Active_subscribers','Suspended_subscribers'],inplace=True)

In [12]:
df

,PID,CRM_PID_Value_Segment,EffectiveSegment,Billing_ZIP,KA_name,Active_subscribers,Total_SUBs,AvgMobileRevenue,AvgFIXRevenue,TotalRevenue,ARPU,CHURN
0,123759242,Bronze,SOHO,6000.0,VM,2,2,40.17,0.0,40.17,NaN,0
1,126145737,Bronze,SOHO,6400.0,VM,3,3,40.17,0.0,40.17,13.39,0
2,123506355,Bronze,SOHO,6000.0,DI,2,5,40.17,0.0,40.17,20.09,0
3,112595585,Bronze,SOHO,4400.0,MT,1,3,40.17,0.0,40.17,40.17,0
4,115097935,Iron,SOHO,4000.0,AD,2,3,40.17,0.0,40.17,20.09,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8448,115901902,Iron,SOHO,4000.0,VU,2,3,56.50,0.0,56.50,28.25,0
8449,115800273,Bronze,SOHO,4006.0,VU,5,5,56.50,0.0,56.50,11.30,0
8450,160049889,Bronze,SOHO,4003.0,VU,3,4,56.50,0.0,56.50,18.83,0
8451,160095203,Bronze,SOHO,4023.0,VU,3,3,56.50,0.0,56.50,18.83,1


In [57]:
df.isnull().sum()


PID                      0
CRM_PID_Value_Segment    0
EffectiveSegment         0
Billing_ZIP              0
KA_name                  0
Active_subscribers       0
Total_SUBs               0
AvgMobileRevenue         0
AvgFIXRevenue            0
TotalRevenue             0
ARPU                     0
CHURN                    0
dtype: int64

In [58]:
df.columns = df.columns.str.strip()

In [59]:
df['CRM_PID_Value_Segment'] = df['CRM_PID_Value_Segment'].fillna(df['CRM_PID_Value_Segment'].mode()[0])
df['Billing_ZIP'] = df['Billing_ZIP'].fillna(df['Billing_ZIP'].mode()[0])
df['ARPU'] = df['ARPU'].fillna(df['ARPU'].median())

In [83]:
df.isnull().sum()

PID                      0
CRM_PID_Value_Segment    0
EffectiveSegment         0
Billing_ZIP              0
KA_name                  0
Active_subscribers       0
Total_SUBs               0
AvgMobileRevenue         0
AvgFIXRevenue            0
TotalRevenue             0
ARPU                     0
CHURN                    0
dtype: int64

In [84]:
le=LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
  df[col]=le.fit_transform(df[col].astype(str))

In [85]:
X=df.drop('CHURN',axis=1)
y=df['CHURN']

In [99]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42,stratify=y)

In [100]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [101]:
scaler = StandardScaler()
X_train_bal = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

In [102]:
log_reg=LogisticRegression(random_state=42, class_weight='balanced')
log_reg.fit(X_train_balanced,  y_train_balanced)
y_pred_re=log_reg.predict(X_test)

C:\Users\CM\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\CM\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [103]:
rf = BalancedRandomForestClassifier(n_estimators=200, random_state=42,class_weight='balanced')
rf.fit(X_train_balanced, y_train_balanced)
y_pred=rf.predict(X_test)

C:\Users\CM\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but BalancedRandomForestClassifier was fitted with feature names
  warnings.warn(


In [108]:
def evaluate_model(y_test, y_pred, model_name):
    print(f'{model_name} Evaluation:')
    print('Accuracy:',round(accuracy_score(y_test,y_pred),3))
    print('Precision:',round(precision_score(y_test,y_pred),3))
    print('Recall:',round(recall_score(y_test,y_pred),3))
    print("Classification report:")
    print(classification_report(y_test,y_pred))

evaluate_model(y_test,y_pred_re,'Logistic Regression')
evaluate_model(y_test,y_pred_rf,'RandomForestClassification')

Logistic Regression Evaluation:
Accuracy: 0.501
Precision: 0.057
Recall: 0.427
Classification report:
              precision    recall  f1-score   support

           0       0.93      0.51      0.66      1581
           1       0.06      0.43      0.10       110

    accuracy                           0.50      1691
   macro avg       0.49      0.47      0.38      1691
weighted avg       0.87      0.50      0.62      1691

RandomForestClassification Evaluation:
Accuracy: 0.935
Precision: 0.0
Recall: 0.0
Classification report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      1581
           1       0.00      0.00      0.00       110

    accuracy                           0.93      1691
   macro avg       0.47      0.50      0.48      1691
weighted avg       0.87      0.93      0.90      1691



C:\Users\CM\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\CM\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\CM\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\CM\anaconda3\Lib\site-packages\sklearn\metrics\_

In [106]:
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=200, random_state=42)
brf.fit(X_train_balanced, y_train_balanced)
y_pred_brf = brf.predict(X_test)
evaluate_model(y_test, y_pred_brf, "Balanced Random Forest")


Balanced Random Forest Evaluation:
Accuracy: 0.935
Precision: 0.0
Recall: 0.0
Classification report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      1581
           1       0.00      0.00      0.00       110

    accuracy                           0.93      1691
   macro avg       0.47      0.50      0.48      1691
weighted avg       0.87      0.93      0.90      1691



C:\Users\CM\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but BalancedRandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\CM\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\CM\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\CM\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no